In [1]:
import torch
import numpy as np
from torchvision import transforms as T
from lightly.transforms import SimCLRTransform, DINOTransform, MAETransform, MoCoV2Transform, utils
from datasets import create_dataset, create_bootstrap_dataloader
from models import *
from tqdm import tqdm
import pytorch_lightning as pl
import os
import copy
import gc
import matplotlib.pyplot as plt


/home/hussein/pytoniec/lib/python3.13/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pytorch_lightning.callbacks import Callback

class SaveAtEpochsCallback(Callback):
    def __init__(self, save_epochs, dirpath="checkpoints"):
        super().__init__()
        self.save_epochs = set(save_epochs)
        self.dirpath = dirpath
        os.makedirs(self.dirpath, exist_ok=True)

    def on_train_epoch_end(self, trainer, pl_module):
        current_epoch = trainer.current_epoch + 1  # epoka 0-based
        if current_epoch in self.save_epochs:
            filename = f"model_epoch_{current_epoch}.ckpt"
            path = os.path.join(self.dirpath, filename)
            trainer.save_checkpoint(path)
            print(f"Zapisano model po epoce {current_epoch}: {path}")

In [ ]:
SEED = 42

def seed_everything(seed: int=42):
    pl.seed_everything(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    ####### Normaly you would also need to seed those generators but `pytorch_lightning` does it in one func
    # random.seed(seed)
    # np.random.seed(seed)
    # torch.manual_seed(seed)
    ######
    torch.cuda.manual_seed(seed) # Don't know if pytorch lightning does this
    torch.cuda.manual_seed_all(seed) # Don't know if pytorch lightning does this
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(SEED)

